In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import calendar as cld
import mysql.connector  
import numpy as np

In [2]:
import glob
import os

Merging csv files (>950) for match specific info

In [3]:
all_files = glob.glob("E:/Portfolio Project/Data/Cricksheet IPL/ipl_csv2/*_info.csv")
li = []
for file in all_files:
    df = pd.read_csv(file, index_col=False, header=None, on_bad_lines='skip', skiprows=[0,1]).drop(0, axis=1).T
    df.columns = df.loc[1]
    df = df.drop([1], axis = 0)
    file_name = str(file).split('\\')[1].split('_')[0]
    df.insert(0, 'Match_ID', file_name)
    df = df.reset_index(drop=True)
    columns = list(df.columns)
    if 'winner_runs' in columns:
        df['Margin'] = df['winner_runs']
        df['Margin_Type'] = 'runs'
        
    elif 'winner_wickets' in columns:
        df['Margin'] = df['winner_wickets']
        df['Margin_Type'] = 'wickets'
        
    elif (('winner' not in columns) and ('outcome' in columns)):
        df['winner'] = None
        df['Margin'] = 0
        df['player_of_match'] = None
        df['Margin_Type'] = 'Tie'
        
    df = df[['Match_ID', 'team', 'season', 'date',
             'venue', 'city', 'toss_winner', 'toss_decision',
             'winner', 'Margin', 'Margin_Type', 'player_of_match']]
    df.set_index('Match_ID')
    li.append(df)
match_info = pd.concat(li, axis=0, ignore_index=True)

Cleaning 'season' column

In [4]:
match_info.loc[match_info['season'] == '2007/08', 'season'] = '2008'
match_info.loc[match_info['season'] == '2009/10', 'season'] = '2010'
match_info.loc[match_info['season'] == '2020/21', 'season'] = '2020'
match_info.season.unique()

array(['2017', '2018', '2019', '2020', '2021', '2022', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype=object)

Checking 'venue' column

In [5]:
match_info.venue.unique()

array(['Rajiv Gandhi International Stadium, Uppal',
       'Maharashtra Cricket Association Stadium',
       'Saurashtra Cricket Association Stadium', 'Holkar Cricket Stadium',
       'M.Chinnaswamy Stadium', 'Wankhede Stadium', 'Eden Gardens',
       'M Chinnaswamy Stadium', 'Feroz Shah Kotla',
       'Punjab Cricket Association IS Bindra Stadium, Mohali',
       'Green Park', 'Punjab Cricket Association IS Bindra Stadium',
       'Rajiv Gandhi International Stadium', 'MA Chidambaram Stadium',
       'Sawai Mansingh Stadium', 'Arun Jaitley Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Sheikh Zayed Stadium', 'Dubai International Cricket Stadium',
       'Sharjah Cricket Stadium',
       'MA Chidambaram Stadium, Chepauk, Chennai',
       'Wankhede Stadium, Mumbai', 'Narendra Modi Stadium, Ahmedabad',
       'Arun Jaitley Stadium, Delhi', 'Zayed Cricket Stadium, Abu Dhabi',
       'Brabourne Stadium, Mumbai', 'Dr DY Patil Sports Academy, Mumbai',
       

Cleaning 'venue' column & removing duplicates

In [6]:
match_info.loc[match_info['venue'] == 'Arun Jaitley Stadium, Delhi', 'venue'] = 'Arun Jaitley Stadium'
match_info.loc[match_info['venue'] == 'Brabourne Stadium, Mumbai', 'venue'] = 'Brabourne Stadium'
match_info.loc[match_info['venue'] == 'Dr DY Patil Sports Academy, Mumbai', 'venue'] = 'Dr DY Patil Sports Academy'
match_info.loc[match_info['venue'] == 'Eden Gardens, Kolkata', 'venue'] = 'Eden Gardens'
match_info.loc[match_info['venue'] == 'M.Chinnaswamy Stadium', 'venue'] = 'M Chinnaswamy Stadium'
match_info.loc[match_info['venue'] == 'Rajiv Gandhi International Stadium, Uppal', 'venue'] = 'Rajiv Gandhi International Stadium'
match_info.loc[match_info['venue'] == 'Wankhede Stadium, Mumbai', 'venue'] = 'Wankhede Stadium'
match_info.loc[match_info['venue'] == 'Zayed Cricket Stadium, Abu Dhabi', 'venue'] = 'Arun Jaitley Stadium'
match_info.loc[match_info['venue'] == 'Feroz Shah Kotla', 'venue'] = 'Arun Jaitley Stadium'
match_info.loc[match_info['venue'].isin(['MA Chidambaram Stadium, Chepauk', 'MA Chidambaram Stadium, Chepauk, Chennai']), 'venue'] = 'MA Chidambaram Stadium'
match_info.loc[match_info['venue'].isin(['Maharashtra Cricket Association Stadium, Pune', 'Subrata Roy Sahara Stadium']), 'venue'] = 'Maharashtra Cricket Association Stadium'
match_info.loc[match_info['venue'].isin(['Punjab Cricket Association Stadium, Mohali', 'Punjab Cricket Association IS Bindra Stadium, Mohali']), 'venue'] = 'Punjab Cricket Association IS Bindra Stadium'
match_info.venue.unique()

array(['Rajiv Gandhi International Stadium',
       'Maharashtra Cricket Association Stadium',
       'Saurashtra Cricket Association Stadium', 'Holkar Cricket Stadium',
       'M Chinnaswamy Stadium', 'Wankhede Stadium', 'Eden Gardens',
       'Arun Jaitley Stadium',
       'Punjab Cricket Association IS Bindra Stadium', 'Green Park',
       'MA Chidambaram Stadium', 'Sawai Mansingh Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Sheikh Zayed Stadium', 'Dubai International Cricket Stadium',
       'Sharjah Cricket Stadium', 'Narendra Modi Stadium, Ahmedabad',
       'Brabourne Stadium', 'Dr DY Patil Sports Academy', 'Newlands',
       "St George's Park", 'Kingsmead', 'SuperSport Park', 'Buffalo Park',
       'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Sardar Patel Stadium, Motera',
       'Barabati Stadium', 'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stad

Filling Empty Cells for column 'city'

In [7]:
match_info[match_info['city'].isnull()].venue.unique()

array(['Dubai International Cricket Stadium', 'Sharjah Cricket Stadium'],
      dtype=object)

In [8]:
match_info.loc[match_info['venue'] == 'Dubai International Cricket Stadium', 'city'] = 'Dubai'
match_info.loc[match_info['venue'] == 'Sharjah Cricket Stadium', 'city'] = 'Sharjah'
match_info[match_info['city'].isnull()].venue.unique()

array([], dtype=object)

Renaming duplicate columns

In [9]:
match_info.columns

Index(['Match_ID', 'team', 'team', 'season', 'date', 'venue', 'city',
       'toss_winner', 'toss_decision', 'winner', 'Margin', 'Margin_Type',
       'player_of_match'],
      dtype='object', name=1)

In [10]:
match_info.columns.values[1] = 'team1'
match_info.columns.values[2] = 'team2'
match_info.columns

Index(['Match_ID', 'team1', 'team2', 'season', 'date', 'venue', 'city',
       'toss_winner', 'toss_decision', 'winner', 'Margin', 'Margin_Type',
       'player_of_match'],
      dtype='object', name=1)

Changing datatypes of columns

In [14]:
match_info['date'] = pd.to_datetime(match_info.date)
match_info['Match_ID'] = pd.to_numeric(match_info.Match_ID)
match_info['season'] = pd.to_numeric(match_info.season)
match_info['Margin'] = pd.to_numeric(match_info.Margin)
match_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950 entries, 0 to 949
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Match_ID         950 non-null    int64         
 1   team1            950 non-null    object        
 2   team2            950 non-null    object        
 3   season           950 non-null    int64         
 4   date             950 non-null    datetime64[ns]
 5   venue            950 non-null    object        
 6   city             950 non-null    object        
 7   toss_winner      950 non-null    object        
 8   toss_decision    950 non-null    object        
 9   winner           932 non-null    object        
 10  Margin           950 non-null    int64         
 11  Margin_Type      950 non-null    object        
 12  player_of_match  932 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(9)
memory usage: 96.6+ KB


Removing duplicates in column 'team1' & 'team2'

In [12]:
match_info.replace('Kings XI Punjab', 'Punjab Kings', inplace = True)
match_info.replace('Rising Pune Supergiant', 'Rising Pune Supergiants', inplace = True)

Saving dataframe to csv file

In [13]:
match_info.to_csv('matches_info.csv')